# Wikipedia Article Quality for US Cities

This notebook will explore the quality of different Wikipedia articles depicting US cities by constructing a dataset that pulls from a variety of sources, including API calls and Excel files.

## Part 1: Getting Article Metadata

To begin this process, I will be borrowing parameters and functions developed by Dr. David W. McDonald, which is free to use under the Creative Commons CC-BY license.

In [514]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' and 'pandas' modules are not a standard Python modules. You will need to install this with pip/pip3 if you do not already have it
import requests
import pandas as pd

In [515]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'mrqs97@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": ""
}


In [516]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


Since we have defined the functions and parameters for the API call that gets article metadata, we will iterate through a CSV file that contains the title of different Wikipedia articles (particularly those pertaining to US cities).

We first load the article titles from the CSV file, then we iterate through the associated DataFrame to run numerous API calls. Since there are over 20,000 article titles, we will be doing a bulk operation of 20 API calls at a time.

In [517]:
# We load a DataFrame from the CSV file depicting Wikipedia article titles
city_articles = pd.read_csv('us_cities_by_state_SEPT.2023.csv')
title_list = city_articles['page_title'].tolist()

In [518]:
cities_data = []
for i in range(0, len(title_list), 20): # We will increment by 20 since each of our calls will involve 20 simultaneous API calls
    sub_list = title_list[i:i+20]
    info = request_pageinfo_per_article("|".join(sub_list))
    cities_data.extend(list(info['query']['pages'].values()))

We have created a table showing the metadata for each article title from the CSV. Below is a snippet of what that looks like:

In [521]:
# The original CSV contained duplicate titles, which means that several rows are identical to each other.
# Therefore, we drop any duplicate rows so as to not over count
city_data = pd.DataFrame.from_dict(cities_data).drop_duplicates()
city_data

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,redirect,new
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,NaN,NaN
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,NaN,NaN
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,NaN,NaN
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,NaN,NaN
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22150,140221,0,"Wamsutter, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1169591845,15315,NaN,NaN
22151,140185,0,"Wheatland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1176370621,20494,NaN,NaN
22152,140245,0,"Worland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166347917,19443,NaN,NaN
22153,140070,0,"Wright, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166334449,12129,NaN,NaN


## Part 2: Calculating ORES Scores

Now that we have metadata on each Wikipedia article, we can input those parameters into another API call to get ORES scores. For background, ORES is a Machine Learning system that rates the quality of an article and assigns a "score" or a "prediction" to a given article.

To calculate the scores, I borrowed parameters and functions from Dr. David W. McDonald, who has let others use this code under the Creative Commons CC-BY License. As a note -- These functions depend on a user-supplied access token. Since I do not want others using my access token, I have omitted mine from this notebook.

In [522]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "mrqs97@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "mrqs97@uw.edu",         # your email address should go here
    'access_token'  : ""}

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

In [523]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


Now that we have defined the functions, we can use the article metadata that was returned from our last API call, and use that to get information on the ORES score. Specifically, we will need to retrieve the latest revision ID for each article, which was automatically returned from our previous API call. This last revision is what the ORES algorithm will use to judge the article quality.

I first create a list to hold the predicted values, and then later on I will append it as a new column to my original dataframe with the article titles.

**Note:** There are 21519 unique articles to iterate through. Since we are calling indiviudal API calls, we have to wait for this to run in its entirety. Most machines will require 5 hours to run the code to completion.

In [266]:
# We create an empty list to contain our predicted values from the ORES algorithm
predictions = []
rev_ids = city_data['lastrevid'].tolist()
for i in range(1, len(rev_ids)):
    print(i) # This print statement is not necessary, but it can be a useful check to see how many iterations we have left
    rev_id = rev_ids[i]
    ORESPred = request_ores_score_per_article(article_revid=int(rev_id),
                                       email_address="mrqs97@uw.edu",
                                       access_token=ACCESS_TOKEN)
    predictions.append(ORESPred['enwiki']['scores'][f"{rev_id}"]['articlequality']['score']['prediction'])

city_data['prediction'] = predictions # We attach a new column containing our predicted values

20243
20244
20245
20246
20247
20248
20249
20250
20251
20252
20253
20254
20255
20256
20257
20258
20259
20260
20261
20262
20263
20264
20265
20266
20267
20268
20269
20270
20271
20272
20273
20274
20275
20276
20277
20278
20279
20280
20281
20282
20283
20284
20285
20286
20287
20288
20289
20290
20291
20292
20293
20294
20295
20296
20297
20298
20299
20300
20301
20302
20303
20304
20305
20306
20307
20308
20309
20310
20311
20312
20313
20314
20315
20316
20317
20318
20319
20320
20321
20322
20323
20324
20325
20326
20327
20328
20329
20330
20331
20332
20333
20334
20335
20336
20337
20338
20339
20340
20341
20342
20343
20344
20345
20346
20347
20348
20349
20350
20351
20352
20353
20354
20355
20356
20357
20358
20359
20360
20361
20362
20363
20364
20365
20366
20367
20368
20369
20370
20371
20372
20373
20374
20375
20376
20377
20378
20379
20380
20381
20382
20383
20384
20385
20386
20387
20388
20389
20390
20391
20392
20393
20394
20395
20396
20397
20398
20399
20400
20401
20402
20403
20404
20405
20406
20407
20408
2040

In [271]:
d

Once again, we show a preview to demonstrate the new column with the article predictions:

In [530]:
city_data

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,redirect,new,prediction
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,NaN,NaN,C
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,NaN,NaN,C
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,NaN,NaN,C
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,NaN,NaN,GA
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,NaN,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22150,140221,0,"Wamsutter, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1169591845,15315,NaN,NaN,GA
22151,140185,0,"Wheatland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1176370621,20494,NaN,NaN,GA
22152,140245,0,"Worland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166347917,19443,NaN,NaN,GA
22153,140070,0,"Wright, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166334449,12129,NaN,NaN,GA


## Part 3: Combining all Data

Now that we have the ORES scores and metadata for each Wikipedia article, we can combine the rest of our data to get information on state population and US Census Bureau regional designations.

### Part 3a: Preparing the Data for Merging

Before we can merge our datasets, we need to create a column `State` by extracting the state from the title name. Unfortunately, as we will see later, many articles from the web crawl do not represent cities or locations, and others do not have the state in the article title. Accounting for these exceptions is non-trivial, so I have decided to exclude them from my analysis later on.

In [532]:
# We create an empty list to contain the extracted state values, then append to the original DataFrame
states = []
for state in city_data['title'].str.split(', '):
    states.append(state[-1])
    
city_data['state'] = states

In [533]:
city_data

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,redirect,new,prediction,state
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,NaN,NaN,C,Alabama
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,NaN,NaN,C,Alabama
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,NaN,NaN,C,Alabama
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,NaN,NaN,GA,Alabama
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,NaN,NaN,C,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22150,140221,0,"Wamsutter, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1169591845,15315,NaN,NaN,GA,Wyoming
22151,140185,0,"Wheatland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1176370621,20494,NaN,NaN,GA,Wyoming
22152,140245,0,"Worland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166347917,19443,NaN,NaN,GA,Wyoming
22153,140070,0,"Wright, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166334449,12129,NaN,NaN,GA,Wyoming


### Part 3b: Loading the US Regions Spreadsheet

Next, we load the data on different US Regions as defined by the US Census Bureau. Then we perform some data processing to clean the data before merging. We show a preview below to verify that the data has been cleaned

In [534]:
# Load the Data from an Excel File
regions = pd.read_excel("US States by Region - US Census Bureau.xlsx")

In [535]:
# Since the data is stored hierarchically, there was a lot of missing data in the cells of the table,
# so we fill those in for each row below them
regions['REGION'].ffill(inplace=True)
regions['DIVISION'].ffill(inplace=True)

regions = regions[~pd.isna(regions['STATE'])] # We filter out any rows that have NaN values to only get information on actual states

In [536]:
regions

,REGION,DIVISION,STATE
2,Northeast,New England,Connecticut
3,Northeast,New England,Maine
4,Northeast,New England,Massachusetts
5,Northeast,New England,New Hampshire
6,Northeast,New England,Rhode Island
7,Northeast,New England,Vermont
9,Northeast,Middle Atlantic,New Jersey
10,Northeast,Middle Atlantic,New York
11,Northeast,Middle Atlantic,Pennsylvania
14,Midwest,East North Central,Illinois


Now we can merge this with our original dataset.

In [537]:
city_data = city_data.merge(regions, how='left', left_on='state', right_on='STATE')

city_data

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,redirect,new,prediction,state,REGION,DIVISION,STATE
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,NaN,NaN,C,Alabama,South,East South Central,Alabama
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,NaN,NaN,C,Alabama,South,East South Central,Alabama
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,NaN,NaN,C,Alabama,South,East South Central,Alabama
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,NaN,NaN,GA,Alabama,South,East South Central,Alabama
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,NaN,NaN,C,Alabama,South,East South Central,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21514,140221,0,"Wamsutter, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1169591845,15315,NaN,NaN,GA,Wyoming,West,Mountain,Wyoming
21515,140185,0,"Wheatland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1176370621,20494,NaN,NaN,GA,Wyoming,West,Mountain,Wyoming
21516,140245,0,"Worland, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166347917,19443,NaN,NaN,GA,Wyoming,West,Mountain,Wyoming
21517,140070,0,"Wright, Wyoming",wikitext,en,en,ltr,2023-10-10T22:36:04Z,1166334449,12129,NaN,NaN,GA,Wyoming,West,Mountain,Wyoming


### Part 3c: Loading the US State Populations Spreadsheet

We repeat a similar process using the estimated populations of US states from the Census Bureau. We load the data, clean it up, and then merge it with our original dataset.

In [541]:
pop_data = pd.read_excel("NST-EST2022-POP.xlsx")

In [542]:
# Since we loaded the raw data from the Excel file, we will need to clean up the column headers by renaming them
pop_data = pop_data.rename(columns={'table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)':'State', 'Unnamed: 1':'Population_April_2020',  'Unnamed: 2':'Population_July_2020', 'Unnamed: 3':'Population_July_2021', 'Unnamed: 4':'Population_July_2022'})

In [543]:
# We filter out any NaN values in our DataFrame to ensure we get consistent data typing for each column
pop_data = pop_data.dropna()

# We then extract the state information from each of the strings in the ['State'] column,
# eliminate the period at the beginning,and replace the original column with our modified values
states = []
for state_list in pop_data['State'].str.split('.'):
    states.append(state_list[-1])
pop_data['State'] = states
pop_data

,State,Population_April_2020,Population_July_2020,Population_July_2021,Population_July_2022
3,United States,331449520,331511512,332031554.0,333287557.0
4,Northeast,57609156,57448898,57259257.0,57040406.0
5,Midwest,68985537,68961043,68836505.0,68787595.0
6,South,126266262,126450613,127346029.0,128716192.0
7,West,78588565,78650958,78589763.0,78743364.0
8,Alabama,5024356,5031362,5049846.0,5074296.0
9,Alaska,733378,732923,734182.0,733583.0
10,Arizona,7151507,7179943,7264877.0,7359197.0
11,Arkansas,3011555,3014195,3028122.0,3045637.0
12,California,39538245,39501653,39142991.0,39029342.0


As you saw from the table above, there are values that are not truly states, like Puerto Rico, as well as regional summaries in the `['State']` column. This is fine, as we will merge it with our original dataset so that only state values in the original dataset will be selected.

In [544]:
city_data = city_data.merge(pop_data, how='left', left_on='state', right_on='State')

### Part 3d: Areas with No Matches

Note that in order to perform our merges, we needed article titles to contain the name of a state. Unfortuantely, not every article contained that information, we we can see below.

In [545]:
unavailable_areas = city_data[pd.isna(city_data['STATE'])]['title'].tolist()

In [546]:
unavailable_areas

['Utqiaġvik',
 'Los Angeles',
 'San Diego',
 'San Francisco',
 '2010 United States census',
 '2020 United States census',
 'Denver',
 'County (United States)',
 'Miami',
 'Atlanta',
 'Echols County',
 'Wailua, Kauai',
 'Indianapolis',
 'New Orleans',
 'Boston',
 'Nantucket',
 'Hyde Park, Boston',
 'Detroit',
 'Minneapolis',
 'Riverview, St. Louis',
 'St. Louis',
 'Las Vegas',
 'Population',
 'Square mile',
 'New York City',
 'American National Standards Institute',
 'Federal Information Processing Standards',
 'Geographic Names Information System',
 'Eastwood, Syracuse',
 'Oklahoma City',
 'Philadelphia',
 'Pittsburgh',
 'Grand Divisions of Tennessee',
 'Salt Lake City',
 'Wikipedia:Citation needed',
 'Seattle',
 'Milwaukee']

As you can see from the list above, each article title described here either refers to a city that is ubiquitous by its name alone (and thus did not have the state in the article title), or the article title did not refer to any particular place or city at all. For instance, the Wikipedia article for "Los Angeles" is titled just that. The article title does not say "Los Angeles, California", which is what my merges assume. Most people will know what you are referring to when you mention "Los Angeles", without even needing to reference the name of the state. A similar pattern shows for many of the other entries, such as "Philadelphia", "Seattle", "Las Vegas", etc. So it makes sense that cities would not always have the name of their state, if they are well known city.

## Part 4: Analysis

Now that we have combined all of our data, we analyze the dataset to answer several questions.

In [547]:
city_data = city_data[['state', 'DIVISION', 'Population_July_2022', 'title', 'lastrevid', 'prediction']]

city_data

,state,DIVISION,Population_July_2022,title,lastrevid,prediction
0,Alabama,East South Central,5074296.0,"Abbeville, Alabama",1171163550,C
1,Alabama,East South Central,5074296.0,"Adamsville, Alabama",1177621427,C
2,Alabama,East South Central,5074296.0,"Addison, Alabama",1168359898,C
3,Alabama,East South Central,5074296.0,"Akron, Alabama",1165909508,GA
4,Alabama,East South Central,5074296.0,"Alabaster, Alabama",1179139816,C
...,...,...,...,...,...,...
21514,Wyoming,Mountain,581381.0,"Wamsutter, Wyoming",1169591845,GA
21515,Wyoming,Mountain,581381.0,"Wheatland, Wyoming",1176370621,GA
21516,Wyoming,Mountain,581381.0,"Worland, Wyoming",1166347917,GA
21517,Wyoming,Mountain,581381.0,"Wright, Wyoming",1166334449,GA


In [548]:
# We save our refined data to a CSV file
city_data.to_csv("wp_scored_city_articles_by_state.csv")

In [549]:
# From here on out, we will filter the data to only focus on
# valid locations (i.e. valid state values)
city_data = city_data[~pd.isna(city_data['DIVISION'])]

### Q1: Highest/Lowest Total Articles Per Capita (State Level)

We will use this data to find the highest and lowest total number of articles for each state, per capita. This means we will calculate the total number of articles for locations in that state and then divide by the population size of said state.

In [551]:
# We are grouping by each state, and then creating a custom aggregation of the Population by
# taking the length (representative of the number of articles) divided by the max (which is just the state population)
# We then sort the values in descending order then get only the top 10 to get the 10 highest per capita states
pd.DataFrame(city_data.groupby('state').Population_July_2022.agg(lambda x: len(x)/max(x)).sort_values(ascending=False).head(10)).rename(columns={'Population_July_2022':'Articles Per Capita'})

,Articles Per Capita
state,
Vermont,0.000507
North Dakota,0.000457
Maine,0.000349
South Dakota,0.000342
Iowa,0.000326
Alaska,0.000202
Pennsylvania,0.000197
Michigan,0.000177
Wyoming,0.000170


Lowest Total

In [554]:
# We perform similar operations to get the lowest per capita states
pd.DataFrame(city_data.groupby('state').Population_July_2022.agg(lambda x: len(x)/max(x)).sort_values(ascending=True).head(10)).rename(columns={'Population_July_2022':'Articles Per Capita'})

,Articles Per Capita
state,
North Carolina,0.000005
Nevada,0.000006
California,0.000012
Arizona,0.000012
Virginia,0.000015
Oklahoma,0.000018
Florida,0.000018
Kansas,0.000021
Maryland,0.000025


### Q2: Highest/Lowest  High Quality Articles Per Capita (State Level)

We will use this data to find the highest and lowest number of high quality articles for each state, per capita. This means we will calculate the total number of high quality articles for locations in that state and then divide by the population size of said state.

In [563]:
# We first designate a column to determine whether or not an article was deemed "High Quality"
city_data['HQ'] = (city_data['prediction'] == 'FA') | (city_data['prediction'] == 'GA')

,state,DIVISION,Population_July_2022,title,lastrevid,prediction,HQ
0,Alabama,East South Central,5074296.0,"Abbeville, Alabama",1171163550,C,False
1,Alabama,East South Central,5074296.0,"Adamsville, Alabama",1177621427,C,False
2,Alabama,East South Central,5074296.0,"Addison, Alabama",1168359898,C,False
3,Alabama,East South Central,5074296.0,"Akron, Alabama",1165909508,GA,True
4,Alabama,East South Central,5074296.0,"Alabaster, Alabama",1179139816,C,False
...,...,...,...,...,...,...,...
21514,Wyoming,Mountain,581381.0,"Wamsutter, Wyoming",1169591845,GA,True
21515,Wyoming,Mountain,581381.0,"Wheatland, Wyoming",1176370621,GA,True
21516,Wyoming,Mountain,581381.0,"Worland, Wyoming",1166347917,GA,True
21517,Wyoming,Mountain,581381.0,"Wright, Wyoming",1166334449,GA,True


In [565]:
# Create a custom column to calculate the High Quality articles per capita
hq_percap = city_data.groupby('state').agg({'Population_July_2022':'max', 'HQ':'sum'})

hq_percap['HQPerCap'] = hq_percap['HQ'] / hq_percap['Population_July_2022']

In [577]:
pd.DataFrame(hq_percap['HQPerCap'].sort_values(ascending=False).head(10))

,HQPerCap
state,
Vermont,0.000070
Wyoming,0.000067
South Dakota,0.000062
West Virginia,0.000060
Montana,0.000049
New Hampshire,0.000045
Pennsylvania,0.000044
Missouri,0.000042
Alaska,0.000042


In [500]:
pd.DataFrame(hq_percap['HQPerCap'].sort_values(ascending=True).head(10))

,HQPerCap
state,
North Carolina,0.000002
Virginia,0.000002
Nevada,0.000002
Arizona,0.000003
California,0.000004
Florida,0.000005
New York,0.000006
Maryland,0.000007
Kansas,0.000007


### Q3: Total/High Quality Articles Per Capita (Division Level)

We now use the data to summarize the articles per capita (both total, and high quality) at the Census division level. To do this accurately, we are first aggregating on the state level, to retrieve state summaries. Then we are aggregating on the division level to get the per capita information.

In [572]:
# We need to aggregate on the state level, and specifcally we need the state population.
# Since all will have the same state population, we just take the maximum.
# We also need the division, which again does not change for any article with that state value, so we take the max
# Then we take the sum of ['HQ'] to get the number of high quality articles, and the count to get the number of total articles
state_aggs = city_data.groupby('state').agg({'Population_July_2022':'max', 'DIVISION':'max', 'HQ':'sum', 'lastrevid':'count'})

# Now that we have unique values per state, we can sum the populations, the high quality articles, and the counts to
# get a summary for the division level
div_aggs = state_aggs.groupby('DIVISION').agg({'Population_July_2022':'sum', 'HQ':'sum', 'lastrevid':'sum'})

div_aggs['Articles Per Capita'] = div_aggs['lastrevid'] / div_aggs['Population_July_2022']

In [573]:
pd.DataFrame(div_aggs['Articles Per Capita'].sort_values(ascending=False))

,Articles Per Capita
DIVISION,
West North Central,0.000181
New England,0.000125
East North Central,0.000101
Middle Atlantic,0.000090
East South Central,0.000078
West South Central,0.000050
Mountain,0.000046
South Atlantic,0.000028
Pacific,0.000024


In [574]:
div_aggs['HQ Articles Per Capita'] = div_aggs['HQ'] / div_aggs['Population_July_2022']

In [575]:
pd.DataFrame(div_aggs['HQ Articles Per Capita'].sort_values(ascending=False))

,HQ Articles Per Capita
DIVISION,
West North Central,0.000032
Middle Atlantic,0.000025
New England,0.000019
East South Central,0.000016
East North Central,0.000015
West South Central,0.000015
Mountain,0.000013
Pacific,0.000009
South Atlantic,0.000008
